In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable

from transform_output_format import get_2D_output, get_4D_output

pd.set_option ('display.max_columns', None)


In [4]:
# Load data from raw_data
data = np.load('../raw_data/baseline.npz', allow_pickle=True)

In [3]:
y_train = pd.read_csv('../raw_data/y_train.csv')

In [5]:
# Converted y_train(df), a vector, into a matrix
y_train.shape

(1845, 10405)

In [6]:
y_train=get_4D_output(y_train)

In [7]:
y_train.shape

(1845, 4, 51, 51)

In [8]:
GHI=data['GHI']
CLS=data['CLS']

In [9]:
X_model=np.zeros((4,4,1,51,51))
for j in range (4):
    for i in range (4):
        for z in range (1):
            if z==0:
                for k in range (51):
                    for h in range (51):
                        X_model[j,i,z,k,h]=GHI[j,i,k+15,h+15]               
            #else:
                #for k in range (51):
                    #for h in range (51):
                        #X_model[j,i,z,k,h]=CLS[j,i,k+15,h+15]
X_model.shape

(4, 4, 1, 51, 51)

In [71]:
y_model=np.zeros((4,4,1,51,51))
for j in range (4):
    for i in range (4):
        for z in range (1):
            for k in range (51):
                    for h in range (51):
                        y_model[j,i,z,k,h]=y_train[j,i,k,h]
y_model.shape

(4, 4, 1, 51, 51)

In [76]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics

def init_model():
    model = models.Sequential()

    model.add(layers.ConvLSTM2D(32, kernel_size=(3,3), padding='same',
                                data_format='channels_first',
                                return_sequences=True, input_shape=(4,1,51,51)))
    #return_sequences=True output the image at each time step
    model.add(layers.Dense(320))
    print(model.output_shape)
    model.add(layers.Reshape((4,32*51*320)))
    model.add(layers.LSTM(64, return_sequences=False))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='relu'))
    
    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='categorical_accuracy', optimizer=adam, metrics=["categorical_accuracy"])
    print(model.summary())
    
    return model

In [77]:
model=init_model()


(None, 4, 32, 51, 320)
Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_25 (ConvLSTM2D)  (None, 4, 32, 51, 51)    38144     
                                                                 
 dense_30 (Dense)            (None, 4, 32, 51, 320)    16640     
                                                                 
 reshape_19 (Reshape)        (None, 4, 522240)         0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                133710080 
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 133,764,929
Trai

In [78]:
model.fit(X_model, y_model, epochs=50, batch_size = 32)

Epoch 1/50


ValueError: in user code:

    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 10